In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="6kfjjO565pfpxqd6YL4S")
project = rf.workspace("skripsie").project("strawberry.00")
dataset = project.version(15).download("yolov8")

In [ ]:
"""  Modify conifg file for YOLO  """


import yaml

with open("/content/strawberry.00-15/data.yaml") as file:
  yaml_file = yaml.safe_load(file)
  yaml_file['test'] = 'strawberry.00-15/test/images'
  yaml_file['path'] = '/content'

with open("/content/strawberry.00-15/data.yaml", "w") as file:
    yaml.safe_dump(yaml_file, file)

In [ ]:
!pip install ultralytics

In [6]:
!mkdir -p strawberry.00-15/valid/images
!mkdir -p strawberry.00-15/valid/labels

!mkdir -p strawberry.00-15/random/images
!mkdir -p strawberry.00-15/random/labels

In [ ]:
import os
import random

random.seed(42)
#set directories

img_directory = str('strawberry.00-15/train/images')
img_target_directory = str('strawberry.00-15/valid/images')
label_directory = str('strawberry.00-15/train/labels')
label_target_directory = str('strawberry.00-15/valid/labels')
data_set_percent_size = float(0.10)

# list of all current images and labels
img_files = [f for f in os.listdir(img_directory)]
img_files.sort()
label_files = [k for k in os.listdir(label_directory)]
label_files.sort()

all_files = list(zip(img_files, label_files))

# select a percent of the files randomly
random_files = random.sample(all_files, int(len(all_files)*data_set_percent_size))

#print(random_files)

# move the randomly selected images by renaming directory

for random_file_name in random_files:
    #print(directory+'/'+random_file_name)
    #print(target_directory+'/'+random_file_name)
    img_name = random_file_name[0]
    label_name = random_file_name[1]
    os.rename(img_directory+'/'+img_name, img_target_directory+'/'+img_name)
    os.rename(label_directory+'/'+label_name, label_target_directory+'/'+label_name)
    continue

"""
#Uncomment for a reduced dataset

img_random_directory = str('strawberry.00-15/random/images')
label_random_directory = str('strawberry.00-15/random/labels')
random_set_percent_size = float(0.50)
img_files = [f for f in os.listdir(img_directory)]
img_files.sort()
label_files = [k for k in os.listdir(label_directory)]
label_files.sort()

new_all_files = list(zip(img_files, label_files))

# select a percent of the files randomly
new_random_files = random.sample(new_all_files, int(len(new_all_files)*random_set_percent_size))
for random_file_name in new_random_files:
    #print(directory+'/'+random_file_name)
    #print(target_directory+'/'+random_file_name)
    img_name = random_file_name[0]
    label_name = random_file_name[1]
    os.rename(img_directory+'/'+img_name, img_random_directory+'/'+img_name)
    os.rename(label_directory+'/'+label_name, label_random_directory+'/'+label_name)
    continue
"""

In [ ]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data=strawberry.00-15/data.yaml \
epochs=100 \
imgsz=640


In [ ]:
!zip -r results.zip runs/

In [ ]:
"""Test set predictions"""

!yolo predict model=runs/detect/train/weights/best.pt source=strawberry.00-15/test/images

In [ ]:
!yolo task=detect \
mode=val \
model=runs/detect/train/weights/best.pt \
data=strawberry.00-15/data.yaml